# YOLOv8 Training on Google Colab

This notebook helps you train your custom YOLOv8 model for Safety Monitoring using the dataset we prepared.

## 1. Install Dependencies
We need `ultralytics` for YOLOv8 and `supervision` for visualization.

In [ ]:
!pip install ultralytics supervision

## 2. Mount Google Drive (Recommended for Large files)
If you are using the FULL dataset (2.5GB), upload to Drive first.
If you are using the **LITE** dataset (48MB), you can skip this cell and upload `colab_dataset_lite.zip` directly to the file uploading area on the left.

In [ ]:
from google.colab import drive
import os
import shutil

try:
    drive.mount('/content/drive')
    # DEFINE WHERE YOU UPLOADED THE FILES IN DRIVE
    drive_folder = '/content/drive/MyDrive/safety_project'
    colab_dest = '/content'

    if os.path.exists(drive_folder):
        print(f"Copying files from {drive_folder} to {colab_dest}...")
        for file in os.listdir(drive_folder):
            if file.startswith('colab_dataset'):
                src = os.path.join(drive_folder, file)
                dst = os.path.join(colab_dest, file)
                shutil.copy(src, dst)
        print("Copy complete.")
except Exception as e:
    print("Drive mount skipped or failed. Assuming direct upload.")

## 3. Prepare Dataset
Auto-detects if you are using the Lite version or the Full Split version.

In [ ]:
import os

dataset_zip = ""

# Check for Lite version first
if os.path.exists("colab_dataset_lite.zip"):
    print("Found colab_dataset_lite.zip! Using LITE dataset.")
    dataset_zip = "colab_dataset_lite.zip"
    
elif os.path.exists("colab_dataset.zip.001"):
    print("Found split zip files. Recombining... this may take a moment.")
    !cat colab_dataset.zip.* > colab_dataset.zip
    if os.path.exists("colab_dataset.zip"):
        print("Recombination successful!")
        dataset_zip = "colab_dataset.zip"
    else:
        print("Error recombination failed.")

elif os.path.exists("colab_dataset.zip"):
    print("Found full colab_dataset.zip!")
    dataset_zip = "colab_dataset.zip"

if dataset_zip:
    print(f"Unzipping {dataset_zip}...")
    !unzip -q {dataset_zip} -d /content/dataset
    print("Unzip complete. Dataset is in /content/dataset")
else:
    print("ERROR: No dataset found. Please upload colab_dataset_lite.zip OR the split .001 files.")

## 4. Verify Dataset Config

In [ ]:
import yaml

yaml_path = '/content/dataset/data.yaml'

if os.path.exists(yaml_path):
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)
        
    print("Original Config:", data)

    # Fix paths to be absolute for Colab just in case
    data['train'] = '/content/dataset/train/images'
    data['val'] = '/content/dataset/valid/images'
    data['test'] = '/content/dataset/test/images'

    with open(yaml_path, 'w') as f:
        yaml.dump(data, f)
        
    print("Updated Config with absolute paths:", data)
else:
    print("Error: data.yaml not found/extracted.")

## 5. Train the Model

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Train the model
results = model.train(
    data='/content/dataset/data.yaml', 
    epochs=50, 
    imgsz=640, 
    batch=16,
    project='/content/runs/train',
    name='ppe_model_colab'
)

## 6. Save Results
Download the trained weights.

In [ ]:
!zip -r /content/ppe_model_results.zip /content/runs/train/ppe_model_colab

from google.colab import files
files.download('/content/ppe_model_results.zip')